# Analyze outliers from free recall visualization
## 1/21/21

In [2]:
import pandas as pd
import numpy as np
import plotly.graph_objs as go
import plotly.plotly as py
import plotly
import os
import glob
import math
import statistics
import matplotlib.pyplot as plt
import seaborn as sns
import pickle
import copy
from random import randrange
from sklearn.metrics import jaccard_score
import random

from sklearn.decomposition import PCA #for cluster analysis
from gensim.models import KeyedVectors #for word embeddings
import matplotlib.pyplot as plt #for plotting
import os #for importing
import pickle #for loading transcripts
from scipy.stats import pearsonr 

# from _DRAFT_20200604_functions import * #includes constants and score function
from tqdm import tqdm_notebook #for progress bar

%autosave 5

Autosaving every 5 seconds


# I. Imports
## A. Participant Data and Summary of Data

In [4]:
filename = '/Users/alexreblando/Documents/Documents - Alex’s MacBook Air/GitHub/data_analysis_ebs/online_experiment/analysis/segment/usable_data'
filename_sums = '/Users/alexreblando/Documents/Documents - Alex’s MacBook Air/GitHub/data_analysis_ebs/online_experiment/analysis/segment/sums_usable_data'
data_dict = pickle.load( open( filename, "rb" ) )
id_dict = pickle.load( open( 'ids_dict', "rb" ) )
sums = pickle.load( open( filename_sums, "rb" ) )
outliers = pickle.load( open( 'outlier_ids', "rb" ) )
len(data_dict)

316

# I. Printing
## A. Print the outlier recalls

In [6]:
for i in range(0, len(outliers)):
    print(data_dict[outliers[i][0]]['story'][0])
    print(data_dict[outliers[i][0]]['p_type'][0])
    print(data_dict[outliers[i][0]]['prolific_id'][0])
    print(data_dict[outliers[i][0]]['recall'][0])
    

12
0
5eeab761609f0d1df85d61bf
Calvin and Jessie are going to the airport to return to the States from their vacation in Jamaica.  They do not have much time to get to the gate, but the security line is moving fairly well up until the TSA-type agents have a shift change that causes a delay.  While held up in line, Jessie impulsively tells Calvin that she wants to break up.  She apologizes for not telling him before the vacation.  She doesn't feel inspired in their relationship, I think.  Calvin accepts this but is hurt.  He says if she's so bored with him then they might as well break up.  They awkwardly head to their gate and the flight attendant scans their tickets annoyed.  The attendant closes the gate behind them.  They are the last to board.  They are both seated in Economy but separate.  They are three rows apart.  Once Jessie goes to sit, she turns to Calvin and awkwardly shakes his hand to wish him a good flight back to the US... and that she wants her philosophy books returned

## B. Print recalls from same story and priming

In [1]:
print('First Outlier: Story 12, Priming 0')

for i in range(0, len(id_dict[12][0])):
    print(i)
    print(data_dict[id_dict[12][0][i][0]]['recall'][0])
    
print('Second Outlier: Story 12, Priming 1')

for i in range(0, len(id_dict[12][1])):
    print(i)
    print(data_dict[id_dict[12][1][i][0]]['recall'][0])
    
print('\n\nThird Outlier: Story 14, Priming 0')

for i in range(0, len(id_dict[14][0])):
    print(i)
    print(data_dict[id_dict[14][0][i][0]]['recall'][0])
    
print('\n\nFourth Outlier: Story 23, Priming 2')

for i in range(0, len(id_dict[23][2])):
    print(i)
    print(data_dict[id_dict[23][2][i][0]]['recall'][0])
    
print('\n\nFifth Outlier: Story 32, Priming 0')

for i in range(0, len(id_dict[32][0])):
    print(i)
    print(data_dict[id_dict[32][0][i][0]]['recall'][0])

First Outlier: Story 12, Priming 0


NameError: name 'id_dict' is not defined

# II. Checking the numbers of Words
## A. wordCount function

In [62]:
def wordCount(sentence, model = 'fasttext'):
    words = preproc(sentence).split(' ')
    if len(words) <= 10:
        c = 1
    else:
        c = 0
    
    return c

## B. Find the participants who had a recall <= 10

In [63]:
keys = [11, 12, 13, 14, 21, 22, 23, 24, 31, 32, 33, 34, 41, 42, 43, 44]

small_r = []

for key in keys:
    for i in range(0,3):
        # Get the number of participants in that priming group for that story
        partic_n = len(recalls[key][i])
        for j in range(0, partic_n):
            this_count = wordCount(recalls[key][i][j][0])
            if this_count == 1:
                small_r.append(recalls[key][i][j][1])

## C. Print these participants' responses

In [64]:
for i in range(0, len(small_r)):
    print('\n\n\n', small_r[i])
    print(data_dict[small_r[i]]['recall'][0])




 /Users/alexreblando/Documents/GitHub/data_analysis_ebs/online_experiment/data/finalver_b6/76/168_1_online_exp_2020-10-20_12h03.01.613
an argument between two people in the restaurant



 /Users/alexreblando/Documents/GitHub/data_analysis_ebs/online_experiment/data/finalver_b5/78/4_1_online_exp_2020-09-28_06h46.56.248
About a girl and her friend. 



 /Users/alexreblando/Documents/GitHub/data_analysis_ebs/online_experiment/data/finalver_b6/76/85_1_online_exp_2020-10-21_00h21.05.606
a restaurant story.




 /Users/alexreblando/Documents/GitHub/data_analysis_ebs/online_experiment/data/finalver_b5/78/13_1_online_exp_2020-09-28_15h40.26.822
client arrive at the airports



 /Users/alexreblando/Documents/GitHub/data_analysis_ebs/online_experiment/data/finalver_b6/76/140_1_online_exp_2020-10-20_20h06.09.967
The story i heard is on investment and partnership



 /Users/alexreblando/Documents/GitHub/data_analysis_ebs/online_experiment/data/finalver_b5/78/95_1_online_exp_2020-09-28_15h37.25.

## D. Make sums without these responses

In [78]:
for i in range(0, len(small_r)):
    prime = data_dict[small_r[i]]['p_type'][0] + 1
    story = data_dict[small_r[i]]['story'][0]
    # Change the sum of all participants in that story count
    sums[story][0] -= 1
    # Change the sum of participants from that prime
    sums[story][prime] -= 1

## E. Make a data_dict without these responses and then export it and new sums

In [80]:
for i in range(0, len(small_r)):
    data_dict.pop(str(small_r[i]), None)
    
pickle.dump( data_dict , open( 'data_dict_nsr', "wb" ) )
pickle.dump( sums , open( 'sums_nsr', "wb" ) )

In [81]:
data_dict['/Users/alexreblando/Documents/GitHub/data_analysis_ebs/online_experiment/data/finalver_b5/78/13_1_online_exp_2020-09-28_15h40.26.822']

KeyError: '/Users/alexreblando/Documents/GitHub/data_analysis_ebs/online_experiment/data/finalver_b5/78/13_1_online_exp_2020-09-28_15h40.26.822'

In [ ]:
/Users/alexreblando/Documents/GitHub/data_analysis_ebs/online_experiment/data/finalver_b5/78/131_1_online_exp_2020-09-29_01h42.48.208